In [51]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms

from hmog import HmogHelper
from tqdm import tqdm

dataset_path = 'preprocessed_data/'

In [52]:
WINDOW_SIZE = 40000  # ms
WINDOW_SIZES = [20000, 40000, 60000, 80000, 100000, 120000, 140000]

In [73]:
class SimpleNeuralNet(nn.Module):
    def __init__(self) -> None:
        super(SimpleNeuralNet, self).__init__()
        
        input_shape = 64
        
        # self.conv1 = nn.Conv1d(in_channels=input_shape, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(input_shape, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=32, out_features=16)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=16, out_features=1)
        self.out = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.out(x)
        
        return x

In [74]:
class MyDataset(Dataset):
    def __init__(self, X, y, transform=None):
        # self.dataset = np.load(file_path)
        self.X = X
        self.y = y
        self.transform = transform
        
        # self.X = self.dataset[:, 1:-1]
        # self.y = self.dataset[:, -1]
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        # if self.transform is not None:
        #     curr_item_x = self.transform(self.X[idx])
        #     curr_item_y = self.transform(self.y[idx])
        #     
        #     return curr_item_x, curr_item_y
    
        return self.X[idx], self.y[idx]

In [75]:
from sklearn.model_selection import train_test_split

whole_dataset = np.load(os.path.join(dataset_path, 'hmog_vectors_with_labels.npy'))
whole_dataset = whole_dataset.astype(np.float32)
X, y = whole_dataset[:, 1:-1], whole_dataset[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# transform = transforms.Compose([transforms.ToTensor()])

train_dataset = MyDataset(X_train, y_train, transform=None)
test_dataset = MyDataset(X_test, y_test, transform=None)

print(len(train_dataset), len(test_dataset))

417497 104375


In [76]:
def train_one_epoch(dataloader, loss_fn, device):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(dataloader):
        inputs, labels = data
        inputs, labels = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32)

        optimizer.zero_grad()

        outputs = model(inputs).reshape(-1)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

In [77]:
model = SimpleNeuralNet()
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
model.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

EPOCHS = 3

for epoch in tqdm(range(EPOCHS)):
    avg_loss = train_one_epoch(train_dataloader, criterion, device)

    running_vloss = 0.0

    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(test_dataloader):
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.to(device, dtype=torch.float32), vlabels.to(device, dtype=torch.float32)
            
            voutputs = model(vinputs).reshape(-1)
            vloss = criterion(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

  0%|          | 0/3 [00:00<?, ?it/s]

  batch 1000 loss: 43.052040670394895
  batch 2000 loss: 41.63147640037537
  batch 3000 loss: 40.07008311080933
  batch 4000 loss: 38.42133229255676
  batch 5000 loss: 36.66354320335388
  batch 6000 loss: 36.94552133846283


 33%|███▎      | 1/3 [00:18<00:36, 18.43s/it]

LOSS train 36.94552133846283 valid 34.85208511352539
  batch 1000 loss: 33.93632242488861
  batch 2000 loss: 14.750022595822811
  batch 3000 loss: 1.1236298621296883
  batch 4000 loss: 0.712294006884098
  batch 5000 loss: 0.6909599542617798
  batch 6000 loss: 0.6909055152535438


 67%|██████▋   | 2/3 [00:36<00:18, 18.32s/it]

LOSS train 0.6909055152535438 valid 0.6465810537338257
  batch 1000 loss: 0.6360754384100438
  batch 2000 loss: 0.6276267566680909
  batch 3000 loss: 0.6271793736815453
  batch 4000 loss: 0.668477309256792
  batch 5000 loss: 0.6753668496310711
  batch 6000 loss: 0.61218160623312


100%|██████████| 3/3 [00:55<00:00, 18.47s/it]

LOSS train 0.61218160623312 valid 0.6540545225143433


In [79]:
  torch.save(model, 'classification_model')